In [1]:
#our goal is to have top K artist recommendations for a user, so that they would 
from __future__ import print_function
import pandas as pd
import implicit
import numpy as np
import scipy
import random
from operator import itemgetter
from tqdm import tqdm_notebook as tqdm

df = pd.read_csv('lastfm_9000_users.csv', na_filter=False)
df = df[df.plays > 10] #our threshold of relevance is 10 plays
df = df.drop(['Unnamed: 0'], axis=1)

## TO DO

BIG THINGS: 
- set-up cross-validation
- calculate precision/recall/AUC using ALL user-pairs, not only k-recommended values
- calculate NDCG 
- calculate ATOP 
- calculate speed performance

CASES TO EVALUATE:
- sparse vs. non-sparse data
- threshold of relevance
- which users should we hold out (those with > 10/20/30 values?)

PERFORMANCE:
- speed up split_train_per_user, by figuring out how to access zero values faster
- speed up precision, recall evaluation

In [2]:
#create sparse matrix from dataframe object
def create_sparse_matrix(data, user_user = True):
    """
    Creates sparse matrix (csr_matrix) out of pandas dataframe.
    
    Parameters: 
    - data: Dataframe of user/artist data
    - user_user: determines whether output will be for user-to-user or item-to-item collaborative filtering
                 if user_user is True, then rows will be items and columns will be users
    
    Returns: 
    - plays_sparse: a sparse csr_matrix
    
    """
    print("Creating sparse matrix...")
    #grab unique users/artist IDS
    users = list(np.sort(data.user_id.unique()))
    artists = list(data.artist_name.unique())
    plays = list(data.plays)

    # user-user set-up
    if (user_user == True):
        rows = data.artist_name.astype('category', categories=artists).cat.codes
        cols = data.user_id.astype('category', categories=users).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(artists),len(users)))

    #item-item set-up
    else:    
        rows = data.user_id.astype('category', categories=users).cat.codes
        cols = data.artist_name.astype('category', categories=artists).cat.codes
        plays_sparse = scipy.sparse.csr_matrix((plays, (rows, cols)), shape=(len(users),len(artists)))
        
        
        
    return plays_sparse

In [3]:
#calculate how sparse the matrix is
def calculate_sparsity(M):
    """
    Calculates how sparse this matrix
    """
    matrix_size = float(M.shape[0]*M.shape[1]) # Number of possible interactions in the matrix
    num_plays = len(M.nonzero()[0]) # Number of items interacted with
    sparsity = 100*(1 - float(num_plays/matrix_size))
    return sparsity

In [4]:
#split train, test using all user pairs
def make_train_all_user_pairs(data, test_pct):
    """
    params:
        data: data set in csr_matrix format
        test_pct: percentage to be test set
    """
    #create copies of dataset for training and test data
    test = data.copy()
    train = data.copy()
    
    #alter train data, masking/holding-out random user-pair values for some users
    nonzero_idx = train.nonzero() #find indices in data where interaction exists
    
    
    nonzero_pairs = zip(nonzero_idx[0], nonzero_idx[1]) #create pairs of (user, item) index
    
    #determine how many user-pair values we need to mask, according to test_pct
    random.seed(0) #for reproducibility
    num_samples = int(np.ceil(test_pct * len(nonzero_pairs)))
    samples = random.sample(nonzero_pairs, num_samples) #sample random number of user-item pairs without replacement
    print(type(samples))
    #get user, item row and column indices
    user_idx = [index[0] for index in samples] 
    item_idx = [index[1] for index in samples] 
    
    train[user_idx,item_idx] = 0 #mask the randomly chosen user-item pairs
    train.eliminate_zeros() #remove zeros in sparse arrays that was made previously
    
    return train, test, list(set(user_idx)), samples #output unique list of user rows that were altered

In [5]:
#split train, test by user only with k interactions
def split_train_test_per_user(data, test_pct, k = 20):
    """
    Create train matrix with masked values and dictionary of test values 
    
    Parameters:
    - data: csr_matrix, assuming matrix is user-user (item as rows, columns as users)
    - test_pct: percentage of items to mask per user
    
    Output:
    - train: masked matrix
    - test: list of tuples of held out data ((user_idx, item_idx), plays)
    """
    random.seed(0) #for reproducibility
    
    train = data.copy().T.tocsr() #transpose to make procedure easier/more intuitive
    
    test = dict() #dict to keep track of masked user-item values
    
    user_count = 0
    
    #for each user in the training set
    for user_idx in tqdm(range(train.get_shape()[0])):

        #get indices of interactions of this user
        nonzero_idx = train[user_idx].nonzero()
        
        #only hold out users that have enough data (greater than k interactions)
        if nonzero_idx[1].shape[0] >= k:
            user_count += 1
            #create list of tuples: interaction index (row, col) with the number of plays
            nonzero_pairs = [((user_idx, item_idx), train[user_idx,item_idx]) for item_idx in nonzero_idx[1]]
            
            #sort tuples descending by value
            nonzero_sorted = sorted(nonzero_pairs, key = itemgetter(1), reverse = True)

            #get top k values, then sample test_pct% randomly from subset
            top_values = nonzero_sorted[0:k]

            #sample random number of item_indexes without replacement
            num_samples = int(np.ceil(test_pct * k))
            samples = random.sample(top_values, num_samples) 

            #append user_idx, item_
            test[user_idx] = [pair[0][1] for pair in samples]
            
            #mask the randomly chosen items of this user
            for pair in samples:
                train[pair[0][0], pair[0][1]] = 0
    
    #remove zeros in sparse arrays to save space
    train.eliminate_zeros()
    
    return train.T.tocsr(), test, user_count #convert matrix back

In [6]:
#calculate how many interactions are masked compared to previous dataset
def pct_masked(original, altered):
    altered_n = altered.nonzero()[0].shape[0]
    original_n = original.nonzero()[0].shape[0]
    return (original_n - altered_n)/float(altered_n)

In [7]:
#used to evaluate model
def evaluate(model, test, M, n_rec = 20):
    """
    Calculate precision/recall
    
    parameters:
    - model: fitted implicit model that will perform recommendations
    - test: list containing tuples that are heldout for each user
    - M: csr_matrix of item-users, used in fit
    - n_rec: how many recommendations the system outputs
    
    
    returns:
    - two numpy arrays containing precision and recall
    """
    precision= list()
    recall= list()

    M_rec = M.T.tocsr() #transpose to recommend
    
    print('Evaluating model...')
    #calculate precision/recall for each user, append results to list
    for user, holdout_items in tqdm(test.items()):
        print(user)
        rec = model.recommend(user, M_rec, N=n_rec, filter_already_liked_items=True)
        rec_items = [pair[0] for pair in rec]
        print(holdout_items)
        print(rec_items)
        #count true positives in recommended items
        tp = float(0)
        for item in holdout_items: 
            if item in rec_items:
                tp += 1
        print(tp)
                
        #calculate precision and recall
        precision.append(tp/n_rec) #fraction of recommendations that are in hold-out set
        recall.append(tp/len(holdout_items)) #fraction of hold-out set that were reocmmended
    
    return np.asarray(precision), np.asarray(recall)

In [8]:
# rec = model.recommend(1, plays_sparse, N=20, filter_already_liked_items=True)
# rec_items = [pair[0] for pair in rec]

# n_rec = 20
# precision = list()
# recall = list()
# #count true positives in recommended items
# tp = float(0)
# for item in test[1]: 
#     if item in set(rec_items):
#         tp += 1

# #calculate precision and recall
# precision.append(tp/n_rec) #fraction of recommendations that are in hold-out set
# recall.append(tp/len(test[1])) #fraction of hold-out set that were reocmmended

# print(precision)
# print(recall)

NameError: name 'model' is not defined

In [10]:
model.recommend(1, plays_sparse, N=20, filter_already_liked_items=True)

[(165, 1.0694361),
 (53, 1.0670478),
 (57, 1.0462584),
 (917, 1.0395373),
 (129, 1.0323918),
 (151, 1.0288764),
 (638, 1.0213826),
 (115, 1.002938),
 (1244, 0.984923),
 (58, 0.9841857),
 (899, 0.9756909),
 (56, 0.97184217),
 (49, 0.96897936),
 (52, 0.9689131),
 (1148, 0.96093655),
 (91, 0.95487946),
 (67, 0.95382965),
 (59, 0.95150745),
 (80, 0.95141965),
 (50, 0.94559896)]

In [9]:
#MAIN SCRIPT
plays_sparse = create_sparse_matrix(df)

#filter out users with < 15 artists/reduce sparsity if needed
print('Matrix Sparsity:', calculate_sparsity(plays_sparse))

#split train,test by masking random values by user
train, test, user_count = split_train_test_per_user(plays_sparse, 0.20, 20)
print("Percentage of original data masked:", pct_masked(plays_sparse, train))
print("Users masked:", user_count)

# Matrix Factorization
model = implicit.als.AlternatingLeastSquares(factors=50)

# K-Nearest Neighbors
# model = implicit.nearest_neighbours.BM25Recommender()

# train model 
print("Fitting model...")
model.fit(train, show_progress=True)

precision, recall = evaluate(model, test, plays_sparse)
print('Mean of Precision:',np.mean(precision)*100,'%')
print('Std of Precision:',np.std(precision)*100,'%')
print('Mean of Recall:',np.mean(recall)*100,'%')
print('Std of Recall:',np.std(recall)*100,'%')

Creating sparse matrix...


/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/Users/TimGimi/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:24: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead


Matrix Sparsity: 99.9078520973


  0%|          | 0/15 [00:00<?, ?it/s]


Percentage of original data masked: 0.0894432763057
Users masked: 8178
Fitting model...


...

[1144, 56, 6895, 4005]
[27, 1996, 326, 53, 40, 1993, 762, 428, 316, 3634, 405, 2166, 398, 1305, 341, 315, 3388, 3103, 86, 2279]
0.0
8499
[4135, 1130, 56, 1145]
[3, 917, 4117, 907, 620, 651, 84, 4134, 804, 4116, 5467, 143, 3857, 151, 318, 8830, 2580, 4120, 2189, 12211]
0.0
8501
[1646, 47871, 579, 3860]
[369, 1190, 651, 8, 2052, 552, 1139, 1585, 9927, 28, 172, 9285, 1700, 572, 1705, 1076, 1147, 2683, 1691, 9759]
0.0
8502
[16981, 642, 1261, 47873]
[928, 1271, 92, 1278, 129, 143, 3629, 328, 1188, 165, 350, 1521, 352, 1301, 4746, 620, 3464, 2663, 726, 862]
0.0
8504
[289, 280, 975, 7045]
[1000, 835, 836, 7561, 5817, 2000, 826, 5194, 264, 365, 8194, 356, 5334, 5181, 3094, 2848, 2483, 1705, 274, 7562]
0.0
8505
[350, 1144, 7419, 1473]
[771, 313, 775, 1147, 2006, 917, 1145, 644, 14, 727, 369, 106, 150, 1008, 638, 390, 326, 1515, 17, 53]
0.0
8506
[638, 149, 900, 59]
[165, 129, 727, 151, 313, 150, 92, 155, 644, 3, 634, 722, 84, 804, 22, 49, 1515, 3672, 1991, 143]
0.0
8507
[6686, 47881, 39561, 2258

[897, 368, 2480, 2890]
[371, 1745, 120, 2483, 2, 2848, 781, 7484, 804, 365, 379, 2000, 34, 373, 1255, 2002, 1134, 8023, 771, 397]
0.0
8579
[503, 6954, 7868, 48106]
[4103, 1182, 889, 881, 2370, 2376, 1199, 880, 7867, 851, 2266, 1275, 1768, 877, 1545, 195, 7073, 322, 509, 7093]
0.0
8580
[7007, 1506, 3297, 150]
[369, 669, 617, 9, 3270, 17, 14, 2279, 313, 276, 347, 3309, 4676, 762, 3324, 3, 862, 847, 518, 3274]
0.0
8581
[56, 644, 651, 917]
[401, 4117, 5994, 5996, 804, 727, 3245, 315, 7095, 3236, 2549, 848, 129, 3246, 3240, 638, 7690, 47, 3238, 8441]
0.0
8582
[1341, 1668, 20928, 4540]
[722, 129, 1544, 952, 352, 946, 126, 2174, 1314, 1388, 2976, 5819, 1188, 6313, 737, 1512, 850, 1122, 939, 634]
0.0
8583
[313, 2474, 1076, 29]
[1435, 726, 47, 384, 556, 952, 1439, 946, 4273, 185, 1188, 101, 3904, 21024, 98, 2158, 562, 555, 1433, 1692]
0.0
8584
[2079, 1165, 9927, 4273]
[2717, 1068, 551, 2720, 2474, 98, 2713, 1683, 4272, 1039, 1444, 2715, 3753, 2470, 2735, 599, 3470, 5473, 562, 2156]
0.0
8585
[12

[16310, 14063, 1776, 1199]
[27, 313, 150, 41, 40, 332, 22, 315, 1182, 129, 5379, 889, 1514, 1382, 1427, 2971, 642, 47, 647, 423]
0.0
8659
[1471, 1255, 1261, 1244]
[390, 793, 115, 651, 917, 1626, 92, 592, 1219, 89, 111, 1388, 726, 165, 133, 70, 1230, 949, 132, 932]
0.0
8660
[2140, 6005, 4859, 41]
[1906, 458, 2346, 33, 2345, 917, 34, 86, 796, 3, 617, 12, 6149, 3100, 2258, 332, 6469, 900, 27, 9]
0.0
8661
[10479, 9684, 2000, 16286]
[7562, 184, 5177, 2258, 457, 377, 365, 3305, 7561, 2890, 1130, 835, 11769, 5194, 1309, 3666, 2843, 5334, 4874, 982]
0.0
8662
[2661, 48347, 584, 2497]
[775, 56, 1002, 1211, 313, 129, 727, 2668, 150, 115, 2797, 1515, 651, 566, 1134, 390, 213, 3, 75, 2517]
0.0
8663
[1661, 1770, 9507, 60]
[625, 1095, 636, 946, 70, 795, 1041, 1033, 133, 1148, 129, 634, 1520, 899, 1424, 1445, 731, 84, 742, 638]
0.0
8664
[1864, 260, 10, 15845]
[7540, 9623, 429, 3213, 3082, 315, 5605, 4734, 57, 7401, 3474, 592, 10722, 10727, 27, 1912, 9470, 7367, 16383, 19580]
0.0
8665
[5729, 293, 23352

[428, 326, 33, 18]
[369, 347, 5607, 3309, 1890, 3686, 11, 323, 862, 2006, 198, 762, 634, 1408, 878, 2687, 9917, 12, 3274, 568]
0.0
8736
[4507, 669, 553, 5346]
[3270, 1372, 371, 679, 1296, 368, 1303, 995, 2006, 1008, 1897, 899, 1030, 224, 724, 397, 4022, 1048, 1714, 214]
0.0
8738
[11051, 11043, 4621, 12789]
[1038, 897, 1033, 4726, 3815, 1147, 907, 679, 765, 2531, 760, 313, 2800, 900, 2, 150, 2422, 804, 58, 315]
0.0
8739
[1900, 5316, 4402, 5302]
[2174, 2224, 1313, 247, 289, 722, 1154, 242, 237, 309, 1887, 594, 292, 2217, 5422, 230, 244, 2507, 2181, 294]
0.0
8740
[184, 1714, 4, 172]
[3305, 4639, 1002, 3089, 826, 2848, 725, 6291, 8228, 3, 2000, 5628, 4441, 7374, 516, 3094, 2846, 4874, 224, 4602]
0.0
8741
[37392, 16968, 13983, 1585]
[192, 3430, 3213, 44, 2680, 280, 9866, 6120, 2755, 7234, 1708, 2993, 2986, 2745, 363, 2274, 7229, 230, 4499, 2984]
0.0
8742
[44013, 9803, 9805, 9350]
[1004, 4993, 1295, 4992, 5460, 3917, 1241, 7330, 3304, 1872, 17769, 1230, 6525, 2405, 8923, 181, 9526, 5455, 176

[949, 8244, 7765, 6327, 6320, 4538, 4320, 10276, 7187, 1127, 3786, 10157, 1791, 1502, 7759, 6311, 7593, 3334, 11352, 1670]
0.0
8826
[75, 2857, 3629, 848]
[334, 1266, 40, 907, 911, 383, 2094, 458, 351, 3219, 335, 164, 810, 719, 200, 1991, 0, 12, 781, 22]
0.0
8827
[2670, 7249, 17008, 2977]
[1050, 186, 79, 434, 1508, 758, 370, 768, 936, 382, 3219, 3256, 3405, 7611, 4334, 1705, 9302, 7838, 1378, 5992]
0.0
8828
[5853, 15404, 3802, 3796]
[1131, 5668, 4235, 5347, 11510, 4237, 3361, 11926, 11934, 13534, 3378, 25286, 3359, 5625, 9364, 4508, 3368, 3356, 4242, 5613]
0.0
8829
[1481, 214, 18778, 242]
[2097, 341, 980, 262, 40, 56, 722, 2345, 84, 775, 2258, 27, 53, 1509, 423, 332, 334, 2567, 719, 12]
0.0
8830
[1241, 3137, 1706, 1900]
[1512, 1887, 2507, 722, 363, 3100, 2139, 6821, 164, 6822, 592, 4524, 814, 3096, 4756, 3103, 2349, 2533, 1394, 3467]
0.0
8831
[9157, 1309, 371, 1266]
[172, 249, 150, 405, 722, 1714, 300, 2258, 804, 56, 1000, 899, 258, 250, 164, 242, 1887, 1334, 3, 846]
0.0
8832
[2541, 123

In [ ]:
#AUC https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
#Primary metric: ROC-based
#calculate Precision, Recall, TPr, FPr, AUC by comparing resulting matrix to test_data
#Secondary metric: DCG

#cross_validate(data, k)
#get k-fold indices on train (mask again)
#for each different k-fold, loop through the indices, masked as train, test as not
#train model
#calculate Precision, Recall, AUC, append result to list
#return list of scores for each fold

#calculate ATOP? DCG? 